In [ ]:
import pandas as pd
from pathlib import Path
import os

RAW = Path("../data/raw/trans_3000p2_list.txt")
OUT = Path("../data/processed")
OUT.mkdir(parents=True, exist_ok=True)

# Quick read (first 200k rows) so we don't wait forever
df_sample = pd.read_csv(RAW, nrows=200_000, low_memory=False)
df_sample.columns = [c.strip().lower().replace(" ", "_") for c in df_sample.columns]

df_sample.head()


In [ ]:
import pandas as pd
from pathlib import Path

RAW = Path("../data/raw/trans_3000p2_list.txt")
OUT = Path("../data/processed")
OUT.mkdir(parents=True, exist_ok=True)

chunksize = 300_000  # adjust up/down depending on your RAM

banks = set()
currencies = set()
formats = set()

# We'll compute daily KPIs from totals to avoid storing all data
daily_accum = {}  # {date: {total_txns, total_amount_paid, flagged_txns}}

for chunk in pd.read_csv(RAW, chunksize=chunksize, low_memory=False):
    # normalize columns
    chunk.columns = [c.strip().lower().replace(" ", "_") for c in chunk.columns]

    # handle duplicate account columns if present
    if "account" in chunk.columns and "account.1" in chunk.columns:
        chunk = chunk.rename(columns={"account": "from_account", "account.1": "to_account"})

    # parse types we need
    chunk["timestamp"] = pd.to_datetime(chunk["timestamp"], errors="coerce")
    chunk["amount_paid"] = pd.to_numeric(chunk["amount_paid"], errors="coerce")
    chunk["is_laundering"] = pd.to_numeric(chunk["is_laundering"], errors="coerce").fillna(0).astype(int)

    # dimensions
    banks.update(chunk["from_bank"].astype(str).unique())
    banks.update(chunk["to_bank"].astype(str).unique())
    currencies.update(chunk["receiving_currency"].astype(str).unique())
    currencies.update(chunk["payment_currency"].astype(str).unique())
    formats.update(chunk["payment_format"].astype(str).unique())

    # daily rollup
    chunk["date"] = chunk["timestamp"].dt.date
    g = chunk.groupby("date", dropna=False).agg(
        total_txns=("timestamp", "size"),
        total_amount_paid=("amount_paid", "sum"),
        flagged_txns=("is_laundering", "sum"),
    )

    for date, row in g.iterrows():
        if date not in daily_accum:
            daily_accum[date] = {"total_txns": 0, "total_amount_paid": 0.0, "flagged_txns": 0}
        daily_accum[date]["total_txns"] += int(row["total_txns"])
        daily_accum[date]["total_amount_paid"] += float(row["total_amount_paid"]) if pd.notna(row["total_amount_paid"]) else 0.0
        daily_accum[date]["flagged_txns"] += int(row["flagged_txns"])

# build outputs
daily = pd.DataFrame([
    {
        "date": d,
        "total_txns": v["total_txns"],
        "total_amount_paid": v["total_amount_paid"],
        "flagged_txns": v["flagged_txns"],
        "pct_flagged": round((v["flagged_txns"] / v["total_txns"] * 100) if v["total_txns"] else 0.0, 2),
    }
    for d, v in daily_accum.items()
]).sort_values("date")

dim_bank = pd.DataFrame({"bank": sorted(banks)})
dim_currency = pd.DataFrame({"currency": sorted(currencies)})
dim_format = pd.DataFrame({"payment_format": sorted(formats)})

# Save for Power BI
daily.to_csv(OUT / "daily_kpis.csv", index=False)
dim_bank.to_csv(OUT / "dim_bank.csv", index=False)
dim_currency.to_csv(OUT / "dim_currency.csv", index=False)
dim_format.to_csv(OUT / "dim_payment_format.csv", index=False)

list(OUT.glob("*.csv"))


In [ ]:
import pandas as pd
from pathlib import Path

OUT = Path("../data/processed")

for f in ["daily_kpis.csv", "dim_bank.csv", "dim_currency.csv", "dim_payment_format.csv"]:
    p = OUT / f
    print(f, "exists:", p.exists(), "size:", p.stat().st_size if p.exists() else None)

pd.read_csv(OUT / "daily_kpis.csv").head()
